# 自然语言识别处理单元
## [拂晓工作室](https://github.com/Errrneist/Alchemist)
* 此程序可用于识别文字中的语言以及深度分析新闻概况

# 参考资料
* [1] [Apple - SFrame实例](https://apple.github.io/turicreate/docs/api/generated/turicreate.SFrame.html#turicreate.SFrame)
* [2] [Apple - Turicreate中关于SFrame.apply()的文档](https://apple.github.io/turicreate/docs/api/generated/turicreate.SFrame.apply.html)
* [3] [Stackoverflow - Python从字符串中删除子字符串](https://stackoverflow.com/questions/31273642/better-way-to-remove-multiple-words-from-a-string)
* [4] [Stackoverflow - 关于SFrame.apply()和Lambda x的实例](https://stackoverflow.com/questions/33028423/how-can-i-use-apply-with-a-function-that-takes-multiple-inputs)
* [5] [楼宇 - 用Python分析《红楼梦》](https://zhuanlan.zhihu.com/p/29209681)
* [6] [CSDN - 使用pyltp包进行中文分词实例](https://blog.csdn.net/sinat_26917383/article/details/77067515)
* [7] [CSDN - NLP+语义分析: 角色标注、篇章分析](https://blog.csdn.net/sinat_26917383/article/details/55683599)
* [8] [CSDN - NLP情感分析简介](https://blog.csdn.net/android_ruben/article/details/78174172)
* [9] [CSDN - 基于机器学习的NLP情感分析（二）—- 分类问题](https://blog.csdn.net/stary_yan/article/details/75330729)
* [10] [PYLTP - pyltp技术文档](http://pyltp.readthedocs.io/zh_CN/latest/api.html#id15)
* [11] [PYLTP - pyltp介绍文档一](https://www.ltp-cloud.com/intro/#introduction )
* [12] [PYLTP - pyltp介绍文档二](http://ltp.readthedocs.io/zh_CN/latest/appendix.html#id5)
* [13] [PYLTP - pyltp深度训练模型](https://pan.baidu.com/share/link?shareid=1988562907&uk=2738088569#list/path=/)


# 导入库

In [13]:
# 导入基本库
import re
import os
import time
import random
import datetime

# 网络获取相关包
import pymysql
import urllib
from bs4 import BeautifulSoup

# 机器学习与大数据框架
import turicreate as tc
import csv

# 自然语言识别框架
import pyltp
from pyltp import SentenceSplitter
from pyltp import Segmentor
from pyltp import Postagger
from pyltp import NamedEntityRecognizer
LTP_DATA_DIR = '../../../LTP_data_v3.4.0/'  # ltp模型目录的路径
pos_model_path = os.path.join(LTP_DATA_DIR, 'pos.model')  # 词性标注模型路径，模型名称为`pos.model`
cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')  # 分词模型路径，模型名称为`cws.model`
ner_model_path = os.path.join(LTP_DATA_DIR, 'ner.model')  # 命名实体识别模型路径，模型名称为`pos.model`
par_model_path = os.path.join(LTP_DATA_DIR, 'parser.model')  # 依存句法分析模型路径，模型名称为`parser.model`
srl_model_path = os.path.join(LTP_DATA_DIR, 'srl')  # 语义角色标注模型目录路径，模型目录为`srl`。注意该模型路径是一个目录，而不是一个文件。

# 载入数据

In [2]:
# 用于从上次停止工作后恢复数据
news_path = '../DataSets/Eastmoney/News_NLP/China/'
news_source = 'CHINA20180702-1906.csv'
news = tc.SFrame(news_path + news_source)

Unable to parse line ""今年以来，货币供应和社会融资增速放慢，债券市场违约频现，地方政府隐性债务风险渐显，表外业务和非标资产违约增加。金融领域中这一系列问题，已成为当前必须厘清和认真对待的重大问..."

1 lines failed to parse correctly

Finished parsing file /Users/hongjunwu/Documents/GitHub/Alchemist/DataSets/Eastmoney/News_NLP/China/CHINA20180702-1906.csv

Parsing completed. Parsed 100 lines in 0.137668 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,int,int,list,str,int,list,list,list]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Unable to parse line ""今年以来，货币供应和社会融资增速放慢，债券市场违约频现，地方政府隐性债务风险渐显，表外业务和非标资产违约增加。金融领域中这一系列问题，已成为当前必须厘清和认真对待的重大问..."

1 lines failed to parse correctly

Finished parsing file /Users/hongjunwu/Documents/GitHub/Alchemist/DataSets/Eastmoney/News_NLP/China/CHINA20180702-1906.csv

Parsing completed. Parsed 376 lines in 0.173884 secs.

In [3]:
# 载入数据
news_path = '../DataSets/Eastmoney/News/China/'
news_source = 'CHINA20180702-1906.csv'
news = tc.SFrame(news_path + news_source)

# 预览
news

Finished parsing file /Users/hongjunwu/Documents/GitHub/Alchemist/DataSets/Eastmoney/News/China/CHINA20180702-1906.csv

Parsing completed. Parsed 100 lines in 0.028013 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,int,int,list,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/hongjunwu/Documents/GitHub/Alchemist/DataSets/Eastmoney/News/China/CHINA20180702-1906.csv

Parsing completed. Parsed 509 lines in 0.044659 secs.

contents,date,day,month,related
小微企业融资难、融资贵问题，是一个世界性难题。甚至，外延更为丰富的普惠金融，在很多时候、在不 ...,20180630,30,6,"[保险, 陆家嘴, 银行]"
据国家统计局30日消息，2018年6月份，中国制造业采购经理指数(PMI)为51.5%，比上 ...,20180630,30,6,[]
针对今年前5个月全国规模以上工业企业利润数据与去年同期数据的差异问题，国家统计局工业司有关负 ...,20180629,29,6,[]
今日，内蒙古自治区住房资金管理中心网站发布《关于进一步规范调整住房公积金提取政策的通知》称， ...,20180702,2,7,[]
自2016年10月份国务院发布《关于市场化银行债权转股权的指导意见》以来，债转股一直存在“落 ...,20180702,2,7,[银行]
污水处理、垃圾处理、节水、节能环保……国家发展改革委2日对外发布《关于创新和完善促进绿色发展 ...,20180702,2,7,[]
29日，中国人民银行会同银保监会、证监会、国家发展改革委、财政部等联合召开全国深化小微企业金 ...,20180630,30,6,[银行]
货币政策委员会二季度例会释放信号——日前召开的中国人民银行货币政策委员会2018年第二季度例 ...,20180702,2,7,[银行]
近日，由中国旅游研究院主办，人民日报客户端旅游频道提供媒体支持的《中国入境旅游发展年度报告2 ...,20180702,2,7,[]
作为银行发起设立的债转股实施机构，金融资产投资公司是一类新型实施机构。从昨日开始，这类机构有 ...,20180630,30,6,[银行]


# 检查数据纯净度
* 测试用，检查清洗数据的效果

In [4]:
# 随机人工检查数据纯净度

def showContents(number_to_show, total_amount):
    counter = 0
    while counter < number_to_show:
        print(news['contents'][int(random.uniform(0, total_amount - 1))])
        print('')
        counter += 1
    
showContents(3, len(news['contents']))

金融领域首提外资股比放宽至51%新版外商投资负面清单落地国家发改委、商务部近日发布《自由贸易试验区外商投资准入特别管理措施(负面清单)(2018年版)》自2018年7月30日起施行。国家发改委、商务部近日发布《自由贸易试验区外商投资准入特别管理措施(负面清单)(2018年版)》自2018年7月30日起施行。2018年版负面清单，大幅度放宽市场准入，清单长度由63条减至48条，在银行、汽车、铁路干线网、电网、加油站等22个领域推出一系列重大开放措施。据国家发改委新闻发言人严鹏程介绍，新版外商投资负面清单最重要的是“缩减”二字。除了公布2018年及未来几年的开放措施，一改以往“一次一放”的开放模式，此次还公布未来几年的开放措施，极大的增强了开放的可预期性。记者梳理发现，近5年来，中国已两次修订《外商投资产业指导目录》，外商投资限制性措施缩减65%，审批时间缩短80%以上。“以更大力度推进对外开放。”近日，国家发改委有关负责人就2018年版外商投资准入特别管理措施(负面清单)答记者问时说，近年来，我国不断加快开放步伐，已经将外商投资准入限制措施减少近三分之二，开放的大门越开越大。对外商投资审批制度进行了重大改革，实行负面清单管理新模式，负面清单之外的领域基本实行备案管理。出台了一系列积极吸引外资措施，构建更加完善的投资环境。根据联合国贸发会议《世界投资报告》，我国在全球最具吸引力的投资目的地排名中保持前两位，也是全球第二大引资国。据国家发改委介绍，当前，我国经济已由高速增长阶段转向高质量发展阶段，国际环境正在发生深刻变化，特别是贸易保护主义抬头。未来中国经济实现高质量发展也必须在更加开放条件下进行，因此需要大幅度放宽市场准入。尤其值得一提的是，2018年版负面清单，大幅度放宽市场准入，尤其是大幅扩大服务业开放，多领域的外资股比首提放宽至51%。本次修订外商投资准入负面清单，服务业开放是重点。金融领域，取消银行业外资股比限制，将证券公司、基金管理公司、期货公司、寿险公司的外资股比放宽至51%，2021年取消金融领域所有外资股比限制。基础设施领域，取消铁路干线路网、电网外资限制。交通运输领域，取消铁路旅客运输公司、国际海上运输、国际船舶代理外资限制。商贸流通领域，取消加油站、粮食收购批发外资限制。文化领域，取消禁止投资互联网上网服务营业场所的规定。2018年版负面清单基本放开

# 自然语言识别

## 第一步 分句分词

### 分句函数
* 将句子分开并创建名为'content_split_sents'的列

In [5]:
# 分句函数
def splitSents(contents):
    return SentenceSplitter.split(contents)

# 执行分句函数
news['content_split_sents'] = news['contents'].apply(splitSents)

# 预览
news

contents,date,day,month,related
小微企业融资难、融资贵问题，是一个世界性难题。甚至，外延更为丰富的普惠金融，在很多时候、在不 ...,20180630,30,6,"[保险, 陆家嘴, 银行]"
据国家统计局30日消息，2018年6月份，中国制造业采购经理指数(PMI)为51.5%，比上 ...,20180630,30,6,[]
针对今年前5个月全国规模以上工业企业利润数据与去年同期数据的差异问题，国家统计局工业司有关负 ...,20180629,29,6,[]
今日，内蒙古自治区住房资金管理中心网站发布《关于进一步规范调整住房公积金提取政策的通知》称， ...,20180702,2,7,[]
自2016年10月份国务院发布《关于市场化银行债权转股权的指导意见》以来，债转股一直存在“落 ...,20180702,2,7,[银行]
污水处理、垃圾处理、节水、节能环保……国家发展改革委2日对外发布《关于创新和完善促进绿色发展 ...,20180702,2,7,[]
29日，中国人民银行会同银保监会、证监会、国家发展改革委、财政部等联合召开全国深化小微企业金 ...,20180630,30,6,[银行]
货币政策委员会二季度例会释放信号——日前召开的中国人民银行货币政策委员会2018年第二季度例 ...,20180702,2,7,[银行]
近日，由中国旅游研究院主办，人民日报客户端旅游频道提供媒体支持的《中国入境旅游发展年度报告2 ...,20180702,2,7,[]
作为银行发起设立的债转股实施机构，金融资产投资公司是一类新型实施机构。从昨日开始，这类机构有 ...,20180630,30,6,[银行]


### 分词函数
* 将词分开 并创建名为'contents_split_words'的列

In [6]:
# 分词函数
def splitWords(contents):
    segmentor = Segmentor()  # 初始化实例
    segmentor.load(cws_model_path)  # 加载模型
    words = segmentor.segment(contents)  # 分词
    segmentor.release()  # 释放模型
    return words

# 执行分词函数
news['content_split_words'] = news['contents'].apply(splitWords)

# 预览
news

contents,date,day,month,related
小微企业融资难、融资贵问题，是一个世界性难题。甚至，外延更为丰富的普惠金融，在很多时候、在不 ...,20180630,30,6,"[保险, 陆家嘴, 银行]"
据国家统计局30日消息，2018年6月份，中国制造业采购经理指数(PMI)为51.5%，比上 ...,20180630,30,6,[]
针对今年前5个月全国规模以上工业企业利润数据与去年同期数据的差异问题，国家统计局工业司有关负 ...,20180629,29,6,[]
今日，内蒙古自治区住房资金管理中心网站发布《关于进一步规范调整住房公积金提取政策的通知》称， ...,20180702,2,7,[]
自2016年10月份国务院发布《关于市场化银行债权转股权的指导意见》以来，债转股一直存在“落 ...,20180702,2,7,[银行]
污水处理、垃圾处理、节水、节能环保……国家发展改革委2日对外发布《关于创新和完善促进绿色发展 ...,20180702,2,7,[]
29日，中国人民银行会同银保监会、证监会、国家发展改革委、财政部等联合召开全国深化小微企业金 ...,20180630,30,6,[银行]
货币政策委员会二季度例会释放信号——日前召开的中国人民银行货币政策委员会2018年第二季度例 ...,20180702,2,7,[银行]
近日，由中国旅游研究院主办，人民日报客户端旅游频道提供媒体支持的《中国入境旅游发展年度报告2 ...,20180702,2,7,[]
作为银行发起设立的债转股实施机构，金融资产投资公司是一类新型实施机构。从昨日开始，这类机构有 ...,20180630,30,6,[银行]


## 第二步 词性标注

### 词性分别标注

In [7]:
def tagWords(contents_split):
    postagger = Postagger() # 初始化实例
    postagger.load(pos_model_path)  # 加载模型

    words = contents_split  # 分词结果
    postags = postagger.postag(words)  # 词性标注

    postagger.release()  # 释放模型
    return postags

In [8]:
# 执行分词函数
news['content_tag_words'] = news['content_split_words'].apply(tagWords)

# 预览
news

contents,date,day,month,related
小微企业融资难、融资贵问题，是一个世界性难题。甚至，外延更为丰富的普惠金融，在很多时候、在不 ...,20180630,30,6,"[保险, 陆家嘴, 银行]"
据国家统计局30日消息，2018年6月份，中国制造业采购经理指数(PMI)为51.5%，比上 ...,20180630,30,6,[]
针对今年前5个月全国规模以上工业企业利润数据与去年同期数据的差异问题，国家统计局工业司有关负 ...,20180629,29,6,[]
今日，内蒙古自治区住房资金管理中心网站发布《关于进一步规范调整住房公积金提取政策的通知》称， ...,20180702,2,7,[]
自2016年10月份国务院发布《关于市场化银行债权转股权的指导意见》以来，债转股一直存在“落 ...,20180702,2,7,[银行]
污水处理、垃圾处理、节水、节能环保……国家发展改革委2日对外发布《关于创新和完善促进绿色发展 ...,20180702,2,7,[]
29日，中国人民银行会同银保监会、证监会、国家发展改革委、财政部等联合召开全国深化小微企业金 ...,20180630,30,6,[银行]
货币政策委员会二季度例会释放信号——日前召开的中国人民银行货币政策委员会2018年第二季度例 ...,20180702,2,7,[银行]
近日，由中国旅游研究院主办，人民日报客户端旅游频道提供媒体支持的《中国入境旅游发展年度报告2 ...,20180702,2,7,[]
作为银行发起设立的债转股实施机构，金融资产投资公司是一类新型实施机构。从昨日开始，这类机构有 ...,20180630,30,6,[银行]


### 为每条新闻创建便于索引的字典（实用性待定 耗时长）

In [19]:
def dictTagWords(words, tags):
    dictionary = {}
    print(len(words), len(tags))
    counter = 0
    while counter < len(words):
        dictionary[words[counter]] = tags[counter]
        counter += 1
    return dictionary

In [20]:
# 执行分词函数
news['content_tag_dict'] = news.apply(lambda x: dictTagWords(news['content_split_words'],news['content_tag_words']))

# 预览
news

509 509


TypeError: unhashable type: 'list'

## 第三步 命名实体识别

In [14]:
# -*- coding: utf-8 -*-
recognizer = NamedEntityRecognizer() # 初始化实例
recognizer.load(ner_model_path)  # 加载模型

words = ['元芳', '你', '怎么', '看']
postags = ['nh', 'r', 'r', 'v']
netags = recognizer.recognize(words, postags)  # 命名实体识别

print('\t'.join(netags))
recognizer.release()  # 释放模型

S-Nh	O	O	O


In [24]:
# 实体识别
def recognizeWords(words, postages):
    recognizer = NamedEntityRecognizer() # 初始化实例
    recognizer.load(ner_model_path)  # 加载模型
    netags = recognizer.recognize(words, postages)  # 命名实体识别
    recognizer.release()  # 释放模型
    return netags

In [26]:
# 执行实体识别函数
news['recognized_words'] = news['content_split_words'].apply(lambda x:recognizeWords(x ,news['content_tag_words']))

# 预览
news

ArgumentError: Python argument types in
    NamedEntityRecognizer.recognize(NamedEntityRecognizer, list, SArray)
did not match C++ signature:
    recognize(NamedEntityRecognizer {lvalue}, boost::python::list, boost::python::list)
    recognize(NamedEntityRecognizer {lvalue}, boost::python::list, std::__1::vector<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, std::__1::allocator<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > > >)
    recognize(NamedEntityRecognizer {lvalue}, std::__1::vector<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, std::__1::allocator<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > > >, boost::python::list)
    recognize(NamedEntityRecognizer {lvalue}, std::__1::vector<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, std::__1::allocator<std::__1::basic_string<char, std::__

## 第四步 依存句法分析

## 第五步 语义角色分析

# 保存数据

In [ ]:
# 保存数据
filepath = '../DataSets/Eastmoney/News_NLP/China/'
news.save(filepath + news_source)

# 打印时间戳
print('程序运行时间戳：20' 
      + str(datetime.datetime.now().strftime("%y")) + '年'
      + str(datetime.datetime.now().strftime("%m")) + '月' 
      + str(datetime.datetime.now().strftime("%d")) + '日' 
      + str(datetime.datetime.now().strftime("%H")) + '时' 
      + str(datetime.datetime.now().strftime("%M")) + '分' 
      + str(datetime.datetime.now().strftime("%S")) + '秒')

# 打印数据路径
print('\n成功保存自然语言识别数据文件！数据路径：')
print(filepath + news_source)